<a href="https://colab.research.google.com/github/Juxhee/DL_tensorflow/blob/master/RNN_%EA%B4%80%EB%A0%A8%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 임베딩


In [0]:
import tensorflow as tf 
print(tf.__version__)


In [0]:
!pip uninstall tensorflow
!pip install tensorflow==1.15

In [0]:
import tensorflow as tf
print(tf.__version__)

In [0]:
import numpy as np

In [0]:
vocab_size = 100 # one-hot encoding된 vocab 크기
embedding_size = 25 # 임베딩된 vocab크기 

In [0]:
# 인풋 데이터 받기 위한 플레이스홀더 선언
inputs = tf.placeholder(tf.int32, shape=[None])

In [0]:
# 인풋 데이터를 변환하기 위한 embedding matrix(100x25)선언
embedding = tf.Variable(tf.random_normal([vocab_size,embedding_size]),dtype=tf.float32)
# tf.nn.embedding_lookup :
# int32나 int64 형태의 스칼라 형태의 인풋 데이터를 vocab 사이즈만큼의 임베딩된 vector로 변환
embedded_inputs = tf.nn.embedding_lookup(embedding,inputs)


In [0]:
# 세션을 열고 모든 변수에 초기값 할당
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [0]:
# tf.nn.embedding_lookup 테스트 케이스 1
input_data = np.array([7])
print("Embedding 전 인풋 데이터 : ")
# shape : [1,100]
print(sess.run(tf.one_hot(input_data, vocab_size)))
print("Embedding 결과 : ")
# shape : [1,25]
print(sess.run([embedded_inputs],feed_dict={inputs:input_data}))
print(sess.run([embedded_inputs],feed_dict={inputs:input_data})[0].shape)
# 임베딩된 차원을 출력

In [0]:
# tf.nn.embedding_lookup 테스트 케이스 2
input_data = np.array([7,11,67,42,21])
print("Embedding 전 인풋 데이터 : ")
# shape : [5,100]
print(sess.run(tf.one_hot(input_data, vocab_size)))
print("Embedding 결과 : ")
# shape : [5,25]
print(sess.run([embedded_inputs],feed_dict={inputs:input_data}))
print(sess.run([embedded_inputs],feed_dict={inputs:input_data})[0].shape)
# 임베딩된 차원을 출력

## 경사도 자르기


In [0]:
# 선형 회귀 모델 정의
W = tf.Variable(tf.random_normal(shape=[1],name="W"))
b = tf.Variable(tf.random_normal(shape=[1],name="b"))
x = tf.placeholder(tf.float32, name="x")
linear_model = W*x + b

In [0]:
# true value를 입력받기 위한 플레이스홀더 정의
y = tf.placeholder(tf.float32, name="y")

# 손실 함수 정의
loss = tf.reduce_mean(tf.square(linear_model - y))

In [0]:
# 최적화를 위한 그라디언트 디센트 옵티마이저 정의

# 원본 코드
'''
optimizer = tf.train.GradientDescentOptimizer(0.01)
train_step = optimizer.minimize(loss)
'''

In [0]:
# gradient clipping 을 적용한 코드
grad_clip = 5
tvars = tf.trainable_variables()
grads, _ = tf.clip_by_global_norm(tf.gradients(loss,tvars),grad_clip)
optimizer = tf.train.GradientDescentOptimizer(0.01)
train_step = optimizer.apply_gradients(zip(grads,tvars))

In [0]:
# 트레이닝을 위한 입력값과 출력값 준비
x_train = [1,2,3,4]
y_train = [2,4,6,8]

# 세션 실행하고 파라미터를 normal distribution 에서 추출한 임의의 값으로 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [0]:
# 경사하강법 1000번 수행
for i in range(1000): 
  sess.run(train_step, feed_dict={x:x_train, y:y_train})

x_test = [3.5,5,5.5,6]
# 테스트 데이터 이용해 측정
print(sess.run(linear_model, feed_dict={x:x_test}))
sess.close()